In [2]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import tkinter as tk
from tkinter import scrolledtext

In [3]:
# Download necessary NLTK resources
nltk.download('punkt')
nltk.download('stopwords')



[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\asus\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\asus\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [4]:
# FAQ Data
faq = {
    "Does the phone have 5G support?": "Yes, the phone supports 5G connectivity.",
    "What is the warranty period of the phone?": "The warranty period is 2 years from the date of purchase.",
    "How do I reset my phone to factory settings?": "To reset your phone, go to Settings > General > Reset > Erase All Content and Settings.",
    "What is the battery life of the phone?": "The battery lasts up to 12 hours on a full charge, depending on usage.",
    "Does the phone support wireless charging?": "Yes, this phone supports wireless charging with a Qi-compatible charger.",
    "Is the phone water-resistant?": "Yes, the phone is IP68 certified for water and dust resistance.", 
    "My phone is overheating, what should I do?": "f your phone is overheating, close unused apps, avoid using it while charging, and remove the case to let it cool down.",
    "Why is my phone not charging?": "Ensure the charging cable and adapter are working and check for debris in the charging port. If the issue persists, contact customer support.",
    "How do I transfer data from my old phone to the new one?": "Use the built-in Phone Transfer app or connect both phones to a computer and use the transfer tool.",
    "Does the phone support fast charging?": "Yes, it supports fast charging with the included charger."
}


In [5]:
# Preprocessing Function
def preprocess_text(text):
    """
    Preprocesses the input by lowering case, tokenizing, removing stopwords,
    and stemming.
    """
    text = text.lower()
    words = word_tokenize(text)
    stop_words = set(stopwords.words('english'))
    words = [word for word in words if word.isalnum() and word not in stop_words]  # Remove non-alphanumeric
    stemmer = PorterStemmer()
    stemmed_words = [stemmer.stem(word) for word in words]
    return ' '.join(stemmed_words)


In [6]:
# Preprocess FAQ Questions
preprocessed_faq = {preprocess_text(q): a for q, a in faq.items()}

# Find Best Match Function
def find_best_match(user_input, faq_data, threshold=0.3):
    """
    Finds the best match for a user's input using TF-IDF and cosine similarity.
    """
    # Vectorize the preprocessed FAQ questions
    vectorizer = TfidfVectorizer()
    faq_questions = list(faq_data.keys())
    faq_answers = list(faq_data.values())
    faq_tfidf = vectorizer.fit_transform(faq_questions)

    # Vectorize the preprocessed user input
    preprocessed_input = preprocess_text(user_input)
    user_input_tfidf = vectorizer.transform([preprocessed_input])

    # Compute cosine similarity
    cosine_similarities = cosine_similarity(user_input_tfidf, faq_tfidf).flatten()
    max_similarity = cosine_similarities.max()

    if max_similarity < threshold:
        return "I’m sorry, I didn’t understand your question. Can you please rephrase?"

    # Find the index of the highest similarity
    best_match_index = cosine_similarities.argmax()
    return faq_answers[best_match_index]


In [7]:
# GUI Implementation
def send_message():
    """
    Gets user input, processes it, and displays the chatbot's response.
    """
    user_input = user_input_entry.get()
    if not user_input.strip():
        return

    chat_window.insert(tk.END, f"You: {user_input}\n")
    user_input_entry.delete(0, tk.END)

    # Get the response
    response = find_best_match(user_input, preprocessed_faq)
    chat_window.insert(tk.END, f"Bot: {response}\n")
    chat_window.yview(tk.END)

In [8]:
# Create the main window
root = tk.Tk()
root.title("FAQ Chatbot")

# Chat window
chat_window = scrolledtext.ScrolledText(root, wrap=tk.WORD, width=50, height=20, state='normal', font=("Arial", 12))
chat_window.grid(row=0, column=0, columnspan=2, padx=10, pady=10)

chat_window.insert(tk.END, "Bot: Hello! I am your FAQ chatbot. How may I assist you today?\n")
chat_window.insert(tk.END, "Bot: You can ask me about warranty, resetting your phone, 5G support, and more.\n")

# Input field
user_input_entry = tk.Entry(root, width=40, font=("Arial", 12))
user_input_entry.grid(row=1, column=0, padx=10, pady=10)

# Send button
send_button = tk.Button(root, text="Send", command=send_message, width=10, font=("Arial", 12))
send_button.grid(row=1, column=1, padx=10, pady=10)

In [9]:
# Run the Application
root.mainloop()